# 23. Introdução ao LangGraph: Fluxos Cíclicos

Até agora, nossas Chains eram lineares (DAGs). Mas e se precisarmos de um **loop**? Ex: "Escreva um código, teste. Se der erro, corrija e teste de novo". Para isso serve o `LangGraph`.

**Objetivos:**
- Entender `StateGraph`, `Nodes` e `Edges`.
- Criar um fluxo com condicional (Router).

In [ ]:
!pip install -qU langchain langchain-openai langchain-community langgraph

In [ ]:
import os
from google.colab import userdata
import getpass

try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Digite sua OpenAI API Key: ")

## 1. Definindo o Estado (State)

O Estado é um dicionário (ou objeto Pydantic) compartilhado entre todos os nós do grafo.

In [ ]:
from typing import TypedDict

class AgenteState(TypedDict):
    pergunta: str
    resposta: str
    revisoes: int

## 2. Definindo os Nós (Nodes)

São funções python simples que recebem o estado e retornam uma atualização para ele.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def gerador(state: AgenteState):
    print(f"--- GERANDO (Tentativa {state['revisoes']}) ---")
    prompt = ChatPromptTemplate.from_template("Responda a pergunta de forma breve: {pergunta}")
    chain = prompt | llm | StrOutputParser()
    res = chain.invoke({"pergunta": state['pergunta']})
    return {"resposta": res, "revisoes": state['revisoes'] + 1}

def critico(state: AgenteState):
    print("--- CRITICANDO ---")
    # Simulação: se a resposta tiver menos de 20 chars, achamos ruim
    # Se já revisou 3 vezes, aceitamos de qualquer jeito para evitar loop infinito
    if len(state['resposta']) < 20 and state['revisoes'] < 3:
        return {"pergunta": state['pergunta'] + " (Seja mais detalhado!)"}
    return {} # Nenhuma mudança

## 3. Definindo a Lógica Condicional (Edges)

Decide se volta para o gerador ou termina.

In [ ]:
from langgraph.graph import END

def router(state: AgenteState):
    # Se a resposta for curta e tivermos revisões sobrando, volta pro gerador
    if len(state['resposta']) < 20 and state['revisoes'] < 3:
        return "gerador"
    return END

## 4. Montando o Grafo

Adicionamos nós e arestas.

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(AgenteState)

workflow.add_node("gerador", gerador)
workflow.add_node("critico", critico)

# Fluxo: Start -> Gerador -> Critico -> Router -> (Gerador ou Fim)
workflow.add_edge(START, "gerador")
workflow.add_edge("gerador", "critico")

workflow.add_conditional_edges(
    "critico",
    router,
    {
        "gerador": "gerador",
        END: END
    }
)

app = workflow.compile()

## 5. Executando

Vamos fazer uma pergunta que exige resposta curta para ver se ele entra no loop.

In [ ]:
inputs = {"pergunta": "Quem descobriu o Brasil?", "revisoes": 0, "resposta": ""}
output = app.invoke(inputs)

print("\n=== FINAL ===")
print(output['resposta'])
print(f"Total revisões: {output['revisoes']}")

## Conclusão

Criamos um sistema que se auto-corrige! O `LangGraph` permite criar fluxos de trabalho resilientes onde o agente pode tentar novamente se falhar.